In [1]:
!cat .env

STAGE=PRODUCTION


In [2]:
Company.objects.all()

<QuerySet [<Company: 574400b8-28b9-4a59-b29f-9d542e42fbdc: OLD UHE-PASSO FUNDO>, <Company: 755f394f-475f-420f-bb4a-e33c5ae754ef: Arteris Planalto Sul - Conserva Teste>, <Company: 57daa1f5-ac75-4a6f-be14-18694a25fdc4: Arteris Régis Bittencourt - Conserva>, <Company: 83ce2755-5d1a-49b6-9fc2-7dbc2cd4ecce: Neovia Rancho Queimado 080/2021>, <Company: b6d9e6b7-c290-4dfc-8902-12d007237f07: Eixo SP - Demonstração>, <Company: 0a3f32bd-b11a-4510-b914-90d3fb3045b9: Vig. Epidemiológica Brusque>, <Company: be11c6d3-a7d3-456a-b0b3-3fa1c5583ca4: Sinaliza Via (Rodovias)>, <Company: 1becd2e8-28f4-4d80-8c94-8112553324b0: Neovia Juquitiba>, <Company: e21922b0-9464-4985-b299-aeb6ef691547: SETA>, <Company: 8c4da155-84b7-43bc-9982-5878eef5b415: Neovia Planalto Sul>, <Company: a5d5d2a0-7063-47d6-9a6d-58080cb65e71: OLD UHE-TESTE>, <Company: 94fc0888-2e78-43c9-ac2d-f6dffc86d5fc: OLD UHE-SALTO SANTIAGO>, <Company: 96bcaf1a-593b-4c6e-bdca-9b0dad0db0f1: OLD UHE-SÃO SALVADOR>, <Company: ffbd0bac-3af9-471c-ade2-23e

In [3]:
import numpy as np
import random
from django.contrib.gis.geos import LineString, Point
from helpers.route_maker import dic_to_ordered_list, unequal_point_pairs
from helpers.strings import keys_to_snake_case, keys_to_camel_case,to_snake_case
from tqdm.notebook import tqdm
import random
import time
import json
from datetime import datetime

In [4]:
to_snake_case('dasdsaTheo')

'dasdsa_theo'

In [5]:
def km_to_coordinates(road, km):

    road_marks = dic_to_ordered_list(road.marks)
    selected_pair = None

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] > km and pair[1]["km"] < km:
            selected_pair = pair
            break
        elif pair[1]["km"] > km and pair[0]["km"] < km:
            selected_pair = pair
            break
        elif pair[0]["km"] == km:
            return Point(pair[0]["point"]["coordinates"]), road
        elif pair[1]["km"] == km:
            return Point(pair[1]["point"]["coordinates"]), road

    # Calculate total length
    start_km = min([selected_pair[0]["km"], selected_pair[1]["km"]])
    end_km = max([selected_pair[0]["km"], selected_pair[1]["km"]])
    segment_mark_length = end_km - start_km
    distance_from_min = km - start_km
    # Check if km is decreasing
    invert_km = selected_pair[1]["km"] <= selected_pair[0]["km"]

    # Cut segment
    start_key = min((selected_pair[0]["index"], selected_pair[1]["index"]))
    end_key = max((selected_pair[0]["index"], selected_pair[1]["index"]))

    points = []
    for key in range(start_key, end_key + 1):
        points.append(
            Point(road.path[key][0], road.path[key][1], road.path[key][2])
        )

    segment = LineString(points, srid=4326)
    # Apply km inversion correction
    if invert_km:
        segment_distance = (
            1 - (distance_from_min / segment_mark_length)
        ) * segment.length
    else:
        segment_distance = (
            distance_from_min / segment_mark_length
        ) * segment.length
    # Find point
    point = segment.interpolate(segment_distance)

    return Point(point.x, point.y), road

In [6]:
def get_connected_reference(company, resource, key):
    select_options = company.custom_options[resource]['fields'][key]['selectOptions']['options']
    return list(a['value'] for a in select_options)

In [7]:
company=Company.objects.get(name="Demo Concessionárias")

In [8]:
lista=list(OccurrenceType.objects.filter(company=company).values_list('name',flat=True))
len(lista)*20


5820

In [9]:
occs=OccurrenceType.objects.filter(company=company,name__in=lista[:5]).order_by('name')
roads=list(Road.objects.filter(company=company).exclude(name__icontains='marcos'))
firm = list(Firm.objects.filter(company=company))
user=User.objects.get(username='rlcs')
status=ServiceOrderActionStatus.objects.get(name='Identificado',companies=company)

occs.count(),user,status,roads

(5,
 <User: rlcs - Kartado CS>,
 <ServiceOrderActionStatus: Identificado - REPORTING_STATUS>,
 [<Road: 884: SP-255>,
  <Road: 885: SPA-334/334>,
  <Road: 886: SP-330>,
  <Road: 887: SP-334>,
  <Road: 888: SP-328>,
  <Road: 889: SP-255>,
  <Road: 890: SP-255>,
  <Road: 891: SP-255>,
  <Road: 892: SP-328>,
  <Road: 893: SP-318>,
  <Road: 894: SPA-196/255>,
  <Road: 895: SPA-348/334>,
  <Road: 896: SPA-198/255>,
  <Road: 897: SPA-004/257>,
  <Road: 898: SP-249>,
  <Road: 899: SPA-384/334>,
  <Road: 900: SPA-397/334>,
  <Road: 901: SPA-020/345>,
  <Road: 902: SPA-307/330>,
  <Road: 903: SPA-043/281>,
  <Road: 904: SPA-268/310>,
  <Road: 905: SPA-011/345>,
  <Road: 906: SPA-077/255>,
  <Road: 907: SPA-318/330>,
  <Road: 908: SPI-274/310>,
  <Road: 909: SPA-334/334>,
  <Road: 910: SPA-338/255>,
  <Road: 911: SPA-179/255>,
  <Road: 912: SPA-352/334>,
  <Road: 913: SP-281>,
  <Road: 914: SPA-074/255>,
  <Road: 915: SPA-115/255>,
  <Road: 916: SPA-133/255>,
  <Road: 917: SPA-138/255>,
  <Road: 

In [10]:
start_date = "2023-01-01"
end_date = "2023-12-31"

start_timestamp = int(time.mktime(datetime.strptime(start_date, "%Y-%m-%d").timetuple()))
end_timestamp = int(time.mktime(datetime.strptime(end_date, "%Y-%m-%d").timetuple()))

In [11]:
ro=np.random.choice(roads)
km=float(np.random.choice(list(r['km'] for i,r in ro.marks.items())))
km

22.0

In [12]:
objects=[]
for a in occs:
    #Quantidade de apontamentos
    for i in range(300):
        
        #rodovia randomica + coordenadas randomicas
        ro=np.random.choice(roads)
        km=float(np.random.choice(list(r['km'] for b,r in ro.marks.items())))
        point, road = km_to_coordinates(ro, km)
        
        #Geração form_data randomico
        form_data={}
        for b in a.form_fields['fields']:
            if b['dataType'] == 'float':
                form_data[to_snake_case(b['apiName'])] = float("{:.2f}".format(np.random.uniform(0, 1)))
            if b['dataType'] == 'number':
                form_data[to_snake_case(b['apiName'])] = int("{:.0f}".format(np.random.uniform(0, 10)))
            if b['dataType'] == 'timestamp':
                #data randomica
                random_timestamp = random.randint(start_timestamp, end_timestamp)
                random_date = datetime.fromtimestamp(random_timestamp)
                form_data[to_snake_case(b['apiName'])] = random_date.strftime("%d-%m-%Y %H:%M")
            if b['dataType'] == 'select':
                form_data[to_snake_case(b['apiName'])] = np.random.choice([c['value'] for c in b['selectOptions']['options']])
            if b['dataType'] == 'selectMultiple':
                form_data[to_snake_case(b['apiName'])] = [np.random.choice([c['value'] for c in b['selectOptions']['options']])]
            if b['dataType'] == 'boolean':
                form_data[to_snake_case(b['apiName'])] = random.choice([True,False])
            if b['dataType'] == 'string' or b['dataType'] == 'textArea':
                form_data[to_snake_case(b['apiName'])] = b['displayName']
        objects.append(Reporting(
                company=company,
                firm=np.random.choice(firm),
                occurrence_type=a,
                lane=np.random.choice(get_connected_reference(company, 'reporting', 'lane')),
                road=road,
                road_name=road.name,
                direction=np.random.choice(get_connected_reference(company, 'reporting', 'direction')),
                created_by=user,
                status=status,
#                     found_at=datetime.datetime(2023, 5, 16, 0, 0).replace(tzinfo=utc),
                km=km,
                end_km_manually_specified=True,
                end_km=float(np.random.choice(list(r['km'] for i,r in road.marks.items()))),
                point=point,
                form_data=form_data
                ))
#     break

In [13]:
for reporting in tqdm(objects):
    reporting.save()
    print(reporting.number)

  0%|          | 0/1500 [00:00<?, ?it/s]

Searching KM on direction...
Searching KM without direction...
Found KM on Road 891: SP-255
DEMO-FDO-Lim-2023.01068
Searching KM on direction...
Searching KM without direction...
Found KM on Road 884: SP-255
DEMO-FDO-Lim-2023.01069
Searching KM on direction...
Searching KM without direction...
Found KM on Road 905: SPA-011/345
DEMO-FDO-Lim-2023.01070
Searching KM on direction...
Searching KM without direction...
Found KM on Road 894: SPA-196/255
DEMO-FDO-Lim-2023.01071
Searching KM on direction...
Searching KM without direction...
Found KM on Road 907: SPA-318/330
DEMO-FDO-Lim-2023.01072
Searching KM on direction...
Searching KM without direction...
Found KM on Road 922: SP-304
DEMO-FDO-Lim-2023.01073
Searching KM on direction...
Found KM on Road 928: SPA-106/255
DEMO-FDO-Lim-2023.01074
Searching KM on direction...
Searching KM without direction...
Found KM on Road 894: SPA-196/255
DEMO-FDO-Lim-2023.01075
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1

Searching KM on direction...
Searching KM without direction...
Found KM on Road 892: SP-328
DEMO-FDO-Lim-2023.01138
Searching KM on direction...
Searching KM without direction...
Found KM on Road 887: SP-334
DEMO-FDO-Lim-2023.01139
Searching KM on direction...
Searching KM without direction...
Found KM on Road 922: SP-304
DEMO-FDO-Lim-2023.01140
Searching KM on direction...
Searching KM without direction...
Found KM on Road 916: SPA-133/255
DEMO-FDO-Lim-2023.01141
Searching KM on direction...
Searching KM without direction...
Found KM on Road 894: SPA-196/255
DEMO-FDO-Lim-2023.01142
Searching KM on direction...
Searching KM without direction...
Found KM on Road 911: SPA-179/255
DEMO-FDO-Lim-2023.01143
Searching KM on direction...
Searching KM without direction...
Found KM on Road 900: SPA-397/334
DEMO-FDO-Lim-2023.01144
Searching KM on direction...
Searching KM without direction...
Found KM on Road 926: SPA-114/255
DEMO-FDO-Lim-2023.01145
Searching KM on direction...
Searching KM witho

DEMO-FDO-Lim-2023.01208
Searching KM on direction...
Searching KM without direction...
Found KM on Road 923: SPA-241/330
DEMO-FDO-Lim-2023.01209
Searching KM on direction...
Searching KM without direction...
Found KM on Road 918: SPA-321/334
DEMO-FDO-Lim-2023.01210
Searching KM on direction...
Searching KM without direction...
Found KM on Road 898: SP-249
DEMO-FDO-Lim-2023.01211
Searching KM on direction...
Searching KM without direction...
Found KM on Road 886: SP-330
DEMO-FDO-Lim-2023.01212
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1632: BR-156
DEMO-FDO-Lim-2023.01213
Searching KM on direction...
Found KM on Road 893: SP-318
DEMO-FDO-Lim-2023.01214
Searching KM on direction...
Searching KM without direction...
Found KM on Road 884: SP-255
DEMO-FDO-Lim-2023.01215
Searching KM on direction...
Searching KM without direction...
Found KM on Road 891: SP-255
DEMO-FDO-Lim-2023.01216
Searching KM on direction...
Searching KM without direction...
Found KM

Searching KM on direction...
Searching KM without direction...
Found KM on Road 921: SPA-271/310
DEMO-FDO-Lim-2023.01279
Searching KM on direction...
Searching KM without direction...
Found KM on Road 927: SP-257
DEMO-FDO-Lim-2023.01280
Searching KM on direction...
Searching KM without direction...
Found KM on Road 896: SPA-198/255
DEMO-FDO-Lim-2023.01281
Searching KM on direction...
Searching KM without direction...
Found KM on Road 904: SPA-268/310
DEMO-FDO-Lim-2023.01282
Searching KM on direction...
Searching KM without direction...
Found KM on Road 911: SPA-179/255
DEMO-FDO-Lim-2023.01283
Searching KM on direction...
Found KM on Road 899: SPA-384/334
DEMO-FDO-Lim-2023.01284
Searching KM on direction...
Searching KM without direction...
Found KM on Road 929: SP-345
DEMO-FDO-Lim-2023.01285
Searching KM on direction...
Searching KM without direction...
Found KM on Road 888: SP-328
DEMO-FDO-Lim-2023.01286
Searching KM on direction...
Searching KM without direction...
Found KM on Road 9

...

Searching KM on direction...
Searching KM without direction...
Found KM on Road 914: SPA-074/255
DEMO-SIN-Tac-2023.01299
Searching KM on direction...
Searching KM without direction...
Found KM on Road 927: SP-257
DEMO-SIN-Tac-2023.01300
Searching KM on direction...
Searching KM without direction...
Found KM on Road 896: SPA-198/255
DEMO-SIN-Tac-2023.01301
Searching KM on direction...
Searching KM without direction...
Found KM on Road 915: SPA-115/255
DEMO-SIN-Tac-2023.01302
Searching KM on direction...
Searching KM without direction...
Found KM on Road 894: SPA-196/255
DEMO-SIN-Tac-2023.01303
Searching KM on direction...
Searching KM without direction...
Found KM on Road 885: SPA-334/334
DEMO-SIN-Tac-2023.01304
Searching KM on direction...
Searching KM without direction...
Found KM on Road 888: SP-328
DEMO-SIN-Tac-2023.01305
Searching KM on direction...
Searching KM without direction...
Found KM on Road 886: SP-330
DEMO-SIN-Tac-2023.01306
Searching KM on direction...
Found KM on Road 9

Searching KM on direction...
Searching KM without direction...
Found KM on Road 898: SP-249
DEMO-default-2023.01149
Searching KM on direction...
Searching KM without direction...
Found KM on Road 894: SPA-196/255
DEMO-default-2023.01150
Searching KM on direction...
Searching KM without direction...
Found KM on Road 898: SP-249
DEMO-default-2023.01151
Searching KM on direction...
Searching KM without direction...
Found KM on Road 927: SP-257
DEMO-default-2023.01152
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1632: BR-156
DEMO-default-2023.01153
Searching KM on direction...
Searching KM without direction...
Found KM on Road 891: SP-255
DEMO-default-2023.01154
Searching KM on direction...
Searching KM without direction...
Found KM on Road 889: SP-255
DEMO-default-2023.01155
Searching KM on direction...
Searching KM without direction...
Found KM on Road 918: SPA-321/334
DEMO-default-2023.01156
Searching KM on direction...
Searching KM without direction..

DEMO-default-2023.01219
Searching KM on direction...
Searching KM without direction...
Found KM on Road 910: SPA-338/255
DEMO-default-2023.01220
Searching KM on direction...
Searching KM without direction...
Found KM on Road 899: SPA-384/334
DEMO-default-2023.01221
Searching KM on direction...
Searching KM without direction...
Found KM on Road 926: SPA-114/255
DEMO-default-2023.01222
Searching KM on direction...
Searching KM without direction...
Found KM on Road 889: SP-255
DEMO-default-2023.01223
Searching KM on direction...
Searching KM without direction...
Found KM on Road 884: SP-255
DEMO-default-2023.01224
Searching KM on direction...
Searching KM without direction...
Found KM on Road 924: SPA-051/255
DEMO-default-2023.01225
Searching KM on direction...
Searching KM without direction...
Found KM on Road 904: SPA-268/310
DEMO-default-2023.01226
Searching KM on direction...
Searching KM without direction...
Found KM on Road 913: SP-281
DEMO-default-2023.01227
Searching KM on directi

Searching KM on direction...
Searching KM without direction...
Found KM on Road 927: SP-257
DEMO-default-2023.01291
Searching KM on direction...
Searching KM without direction...
Found KM on Road 902: SPA-307/330
DEMO-default-2023.01292
Searching KM on direction...
Searching KM without direction...
Found KM on Road 910: SPA-338/255
DEMO-default-2023.01293
Searching KM on direction...
Searching KM without direction...
Found KM on Road 893: SP-318
DEMO-default-2023.01294
Searching KM on direction...
Searching KM without direction...
Found KM on Road 910: SPA-338/255
DEMO-default-2023.01295
Searching KM on direction...
Searching KM without direction...
Found KM on Road 927: SP-257
DEMO-default-2023.01296
Searching KM on direction...
Searching KM without direction...
Found KM on Road 884: SP-255
DEMO-default-2023.01297
Searching KM on direction...
Found KM on Road 927: SP-257
DEMO-default-2023.01298
Searching KM on direction...
Found KM on Road 923: SPA-241/330
DEMO-default-2023.01299
Sear

DEMO-default-2023.01362
Searching KM on direction...
Searching KM without direction...
Found KM on Road 894: SPA-196/255
DEMO-default-2023.01363
Searching KM on direction...
Searching KM without direction...
Found KM on Road 889: SP-255
DEMO-default-2023.01364
Searching KM on direction...
Searching KM without direction...
Found KM on Road 904: SPA-268/310
DEMO-default-2023.01365
Searching KM on direction...
Searching KM without direction...
Found KM on Road 923: SPA-241/330
DEMO-default-2023.01366
Searching KM on direction...
Searching KM without direction...
Found KM on Road 911: SPA-179/255
DEMO-default-2023.01367
Searching KM on direction...
Searching KM without direction...
Found KM on Road 919: SPA-214/255
DEMO-default-2023.01368
Searching KM on direction...
Searching KM without direction...
Found KM on Road 885: SPA-334/334
DEMO-default-2023.01369
Searching KM on direction...
Searching KM without direction...
Found KM on Road 885: SPA-334/334
DEMO-default-2023.01370
Searching KM 

In [8]:
random_timestamp = random.randint(start_timestamp, end_timestamp)
random_date = datetime.fromtimestamp(random_timestamp)

json_data = json.dumps({"date": random_date.strftime("%Y-%m-%d %H:%M:%S")})
print(json_data)

{"date": "2023-02-12 22:13:19"}


  0%|          | 0/80 [00:00<?, ?it/s]

041fd8cc-1980-4f49-b4d7-1a0e7329a16a
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
0d654c80-1fa4-4f29-a685-c4135e048f2d
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2038: BR-116 SP
8feb4bee-8674-4bee-8ffe-eef38659becc
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
12bdc832-ee42-45f6-b186-b2e9f61a8721
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2041: BR-101 RJ
c49dd1fe-8f33-4715-bdb9-6a88d085ab5d
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
445a21ee-7a13-49c8-9fed-6802c4a2d1ef
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
94e81b83-0eb4-449f-8f65-32e07d9ae437
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2036: BR-101 SP
a8c0cf08-d4be-4698-a271-26045c9874ef
Searching KM on direction...
Sea

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
320bafde-63fb-4f04-b98d-2827a1b2a857
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2036: BR-101 SP
d8716dc2-a6b5-496c-909a-60715a3f0606
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2041: BR-101 RJ
7c468793-e120-4b77-9d82-2b00f5cae98d
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2038: BR-116 SP
5a229617-78a7-48ab-8270-08b10ff5d8ab
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
3a698e83-6daa-41eb-bda6-f1e2329e6442
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
d2d3a1d8-82ba-4e06-b533-65bb45ca181a
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
1301aaa7-1727-4bb8-b61d-4d551f76b26a
Searching KM on direction...
Found KM on Road 2037: BR-116 RJ
d71cd562-6

In [7]:
a.__dict__

{'_state': <django.db.models.base.ModelState at 0x7fbdc15d9790>,
 'uuid': UUID('0de2ccfa-92ad-4337-b65f-ad0974b6b87c'),
 'number': 'CCR-DEF-2023.01316',
 'company_id': UUID('3af64f25-59e8-446b-bb1e-963549090b0d'),
 'road_name': 'BR-116 RJ',
 'road_id': 2,
 'km': 270.0,
 'end_km': 325.0,
 'km_reference': None,
 'project_km': 0.0,
 'project_end_km': 0.0,
 'point': <Point object at 0x7fbdc15d8b50>,
 'direction': '4',
 'lane': '1',
 'track': None,
 'branch': None,
 'address': {},
 'created_by_id': UUID('508a66f3-3f58-4c4e-bc9a-fbba6078c02d'),
 'firm_id': UUID('c51e2071-67ca-44ff-ad4a-4a005cf95591'),
 'occurrence_type_id': UUID('0e28ebc6-bf8c-453c-875b-719c3261351c'),
 'form_data': {'year': 0.99,
  'color': '1',
  'index': 3,
  'notes': 'Observações',
  'other': 'Outra interferência',
  'semester': '1',
  'condition': '2',
  'fifth_lca': 0.13,
  'fifth_zpa': 0.85,
  'first_lca': 0.53,
  'first_zpa': 0.26,
  'ninth_lca': 0.41,
  'ninth_zpa': 0.03,
  'sixth_lca': 0.66,
  'sixth_zpa': 0.21,
  

In [12]:
for a in Reporting.objects.filter(occurrence_type__name='Monitoração de Sinalização Vertical',company=company):
    form_data={}
    for b in a.occurrence_type.form_fields['fields']:
            if b['dataType'] == 'float':
                form_data[to_snake_case(b['apiName'])] = float("{:.2f}".format(np.random.uniform(0, 1)))
            if b['dataType'] == 'number':
                form_data[to_snake_case(b['apiName'])] = int("{:.0f}".format(np.random.uniform(0, 10)))
            if b['dataType'] == 'timestamp':
                #data randomica
                random_timestamp = random.randint(start_timestamp, end_timestamp)
                random_date = datetime.fromtimestamp(random_timestamp)
                form_data[to_snake_case(b['apiName'])] = random_date.strftime("%d-%m-%Y %H:%M")
            if b['dataType'] == 'select':
                form_data[to_snake_case(b['apiName'])] = np.random.choice([c['value'] for c in b['selectOptions']['options']])
            if b['dataType'] == 'selectMultiple':
                form_data[to_snake_case(b['apiName'])] = [np.random.choice([c['value'] for c in b['selectOptions']['options']])]
            if b['dataType'] == 'boolean':
                form_data[to_snake_case(b['apiName'])] = random.choice([True,False])
            if b['dataType'] == 'string' or b['dataType'] == 'textArea':
                form_data[to_snake_case(b['apiName'])] = b['displayName']
    a.form_data=form_data
    a.save()

Searching KM on direction...
Found KM on Road 2038: BR-116 SP
Searching KM on direction...
Found KM on Road 2038: BR-116 SP
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2038: BR-116 SP
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2038: BR-116 SP
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2038: BR-116 SP
Searching KM on direction...
Searching KM with